In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
input_dim = 28 * 28
classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, input_dim) / 255.
X_test = X_test.reshape(-1, input_dim) / 255.
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


y_train_not_encoded = y_train
y_test_not_encoded = y_test

y_train = to_categorical(y_train, num_classes=classes)
y_test = to_categorical(y_test, num_classes=classes)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
embed_test = tsne.fit_transform(X_test)

In [ ]:
colors = [
           "#5D2E8C","#FF3D3D","#2EC4B6","#FF9770","#65B891",
           "#3A445D","#0E0F19","#CB9CF2","#0FA3B1","#DC136C"]
palette = sns.color_palette(colors)

In [ ]:
f, ax = plt.subplots(1,1, dpi=144)
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                hue=y_test_not_encoded,
                style=y_test_not_encoded,
                ax=ax,
                palette=palette,
                edgecolor="#F4F4F6",
                markers=["o" for _ in range(10)])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

regressor = LogisticRegression()
regressor.fit(X_train, y_train_not_encoded)

y_pred = regressor.predict(X_test)
accuracy_score(y_pred, y_test_not_encoded)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

regressor = LinearRegression()
regressor.fit(X_train, y_train_not_encoded)

y_pred = regressor.predict(X_test)
y_pred = np.round(y_pred).astype(np.int)
accuracy_score(y_pred, y_test_not_encoded)

In [ ]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(392, input_dim=input_dim, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=32)

In [ ]:
f, ax = plt.subplots(3,1,figsize=(8,15), dpi=144)
pal_tr = sns.color_palette(["#D52941","#ffffff"])
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                ax=ax[0],
                hue= y_test_not_encoded,
                style= y_test_not_encoded,
                palette=palette,
                edgecolor="#F4F4F6",
                markers=["o" for _ in range(10)])
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                ax=ax[1],
                hue=y_pred,
                style=y_pred,
                palette=palette,
                edgecolor="#F4F4F6",
                markers=["o" for _ in range(10)])
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                ax=ax[2],
                hue=y_pred == y_test_not_encoded,
                style=y_pred == y_test_not_encoded,
                palette=pal_tr,
                edgecolor="#000000",
                markers=["X","o"])

In [ ]:
from sklearn.metrics import confusion_matrix
f, ax = plt.subplots(1,1, dpi=121)
mat = confusion_matrix(y_test_not_encoded, y_pred)
sns.heatmap(mat, annot=True, fmt="d", ax=ax)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation="softmax"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=256, epochs=10, validation_split=0.1)

In [ ]:
model.evaluate(X_test, y_test, batch_size=256)

In [ ]:
!git clone https://github.com/otenim/TensorFlow-OS-ELM

In [ ]:
!cp TensorFlow-OS-ELM/os_elm.py ./

In [ ]:
from os_elm import OS_ELM

In [ ]:
def softmax(a):
    c = np.max(a, axis=-1).reshape(-1, 1)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a, axis=-1).reshape(-1, 1)
    return exp_a / sum_exp_a

In [ ]:
hidden_dim = 7000
elm = OS_ELM(
        n_input_nodes= input_dim,
        n_hidden_nodes= hidden_dim,
        n_output_nodes= classes,
        loss='categorical_crossentropy',
        activation='sigmoid')

border = int(1.5 * hidden_dim)
X_train_init = X_train[:border]
X_train_seq = X_train[border:]
y_train_init = y_train[:border]
y_train_seq = y_train[border:]

In [ ]:
pbar = tqdm.tqdm(total=len(X_train), desc='initial training phase')
elm.init_train(X_train_init, y_train_init)
pbar.update(n=len(X_train_init))

pbar.set_description('sequential training phase')
batch_size = 64
for i in range(0, len(X_train_seq), batch_size):
    X_batch = X_train_seq[i:i+batch_size]
    y_batch = y_train_seq[i:i+batch_size]
    elm.seq_train(X_batch, y_batch)
    pbar.update(n=len(X_batch))
pbar.close()

In [ ]:
res = elm._OS_ELM__sess.run([elm._OS_ELM__loss,elm._OS_ELM__accuracy], feed_dict={elm._OS_ELM__x: X_test, elm._OS_ELM__t: y_test})
print("accuracy %f" % res[1])

In [ ]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as k

img_dim = (28, 28)
if k.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_dim[0], img_dim[1])
    X_test = X_test.reshape(X_test.shape[0], 1, img_dim[0], img_dim[1])
    input_shape = (1, img_dim[0], img_dim[1])
else:
    X_train = X_train.reshape(X_train.shape[0], img_dim[0], img_dim[1], 1)
    X_test = X_test.reshape(X_test.shape[0], img_dim[0], img_dim[1], 1)
    input_shape = (img_dim[0], img_dim[1], 1)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=256, epochs=15, validation_split=0.1)

In [ ]:
model.evaluate(X_test, y_test, batch_size=256)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=256)

In [ ]:
f, ax = plt.subplots(3,1,figsize=(8,15), dpi=144)
pal_tr = sns.color_palette(["#D52941","#ffffff"])
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                ax=ax[0],
                hue= y_test_not_encoded,
                style= y_test_not_encoded,
                palette=palette,
                edgecolor="#F4F4F6",
                markers=["o" for _ in range(10)])
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                ax=ax[1],
                hue=y_pred,
                style=y_pred,
                palette=palette,
                edgecolor="#F4F4F6",
                markers=["o" for _ in range(10)])
sns.scatterplot(x=embed_test[:, 0], y=embed_test[:, 1],
                ax=ax[2],
                hue=y_pred == y_test_not_encoded,
                style=y_pred == y_test_not_encoded,
                palette=pal_tr,
                edgecolor="#000000",
                markers=["X","o"])

In [ ]:
from sklearn.metrics import confusion_matrix
f, ax = plt.subplots(1,1, dpi=121)
mat = confusion_matrix(y_test_not_encoded, y_pred)
sns.heatmap(mat, annot=True, fmt="d", ax=ax)